In [11]:
from sklearn.naive_bayes import GaussianNB, MultinomialNB
from sklearn.model_selection import  train_test_split
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score
import json
from sklearn.decomposition import PCA
import os
import pickle
from sklearn.metrics import f1_score
from tqdm import tqdm

In [12]:
config = json.load(open("../../configure.json"))
amzn_2grams = []
amzn_3grams = []
amzn_ngrams = []
aapl_2grams = []
aapl_3grams = []
aapl_ngrams = []

In [13]:
path = config["project_dir"] + config["feature_pkls"]
pkls = os.listdir(path)
results = []
for pkl in tqdm(pkls):
    data = pd.read_pickle(path + pkl, compression='infer')
    train_x = np.stack(data['features'])
    train_y = np.stack(data['trend'].astype('int'))
    names = pkl.split("_", )
    pca = PCA(n_components=296)
    pComponents = pca.fit_transform(train_x)
    x_train, x_test, y_train, y_test = train_test_split(train_x, train_y, test_size=0.2)
    nb = MultinomialNB().fit(x_train,y_train)
    y_pred = nb.predict(x_test)
    acc = accuracy_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    pkl_save_path = config['project_dir'] + "models/nb/pkls/ngram/"
    model_name = "nb_" + pkl
    with open(pkl_save_path + model_name, 'wb') as f:
        pickle.dump(nb, f)
    company = "amazon" if names[0] == "amzn" else "apple"
    result = [names[1], round(np.mean(acc) * 100, 2), round(np.mean(f1) * 100, 2)]
    if "amzn" == names[0]:
        if "2" in names[2]:
            amzn_2grams.append(result)
        if "3" in names[2]:
            amzn_3grams.append(result)
        if "n" in names[2]:
            amzn_ngrams.append(result)
    if "aapl" == names[0]:
        if "2" in names[2]:
            aapl_2grams.append(result)
        if "3" in names[2]:
            aapl_3grams.append(result)
        if "n" in names[2]:
            aapl_ngrams.append(result)

100%|██████████| 36/36 [03:37<00:00,  6.04s/it]


In [14]:
print("amazon 2grams results\n", amzn_2grams)
print("amazon 3grams results\n", amzn_3grams)
print("apple 2grams results\n", aapl_2grams)
print("apple 3grams results\n", aapl_3grams)
print("amazon n grams results\n", amzn_ngrams)
print("apple n grams results\n", aapl_ngrams)


amazon 2grams results
 [['30', 62.41, 41.1], ['60', 61.0, 45.91], ['240', 57.92, 39.03], ['5', 64.52, 32.27], ['1440', 62.68, 45.36], ['15', 61.53, 52.24]]
amazon 3grams results
 [['60', 61.27, 36.42], ['30', 57.92, 28.23], ['240', 58.36, 54.82], ['15', 54.84, 24.67], ['1440', 58.27, 37.96], ['5', 60.04, 35.69]]
apple 2grams results
 [['15', 56.75, 56.24], ['30', 57.61, 52.94], ['60', 58.27, 54.13], ['240', 56.48, 53.22], ['1440', 59.26, 54.25], ['5', 58.97, 47.02]]
apple 3grams results
 [['15', 53.37, 30.63], ['1440', 53.55, 43.52], ['240', 51.72, 54.86], ['60', 56.14, 34.3], ['30', 55.59, 38.79], ['5', 58.27, 28.79]]
amazon n grams results
 [['15', 60.65, 53.68], ['5', 65.76, 35.7], ['1440', 58.19, 26.58], ['240', 58.71, 39.17], ['60', 59.07, 60.49], ['30', 61.44, 25.26]]
apple n grams results
 [['5', 59.39, 47.88], ['240', 56.69, 53.42], ['60', 57.72, 59.59], ['1440', 60.49, 50.2], ['30', 58.45, 50.53], ['15', 53.42, 54.87]]
